In my last post I used Facebook's Llama for creating vector embeddings from different words and sentences. My editor, ChatGPT of course, suggested I use a model that specializes in embedding rather than Llama, which does text generation (chatting). I did, and the results are a bit surprising.

## Choosing models and sentences

The models I want to compare are the Llama3.1 model I used in my previous post, and the models mentioned in Ollama's [blog post about embeddings](https://ollama.com/blog/embedding-models).


In [25]:
from sklearn.metrics.pairwise import cosine_similarity
from langchain_ollama import OllamaEmbeddings
import pandas as pd
from great_tables import GT, style, loc

In [26]:
models = [
    'all-minilm',
    'mxbai-embed-large',
    'nomic-embed-text',
    'llama3.1'
]

I want to calculate the similarities between three sentences, two which sound very different but mean something similar, a person goes into politics, and one sentence which is about public transportation.

In [27]:
sentences = [
    "The man with the tie ran for office.",
    "The woman in the dress became a politician.",
    "The man with the tie ran for a bus."
]

## The results

Next, we loop through the models and use each one of them to create a vector, and then we compare the vectors with each other to create a cosine similarity matrix. 

In [30]:
#| code-fold: true
#| code-summary: "Show the code"

def compare_words(words, embeddings, model_name):
    vectors = []
    for s in words:
        vectors.append(embeddings.embed_query(s))
    matrix = cosine_similarity(vectors)
    df = (
        pd.DataFrame(matrix)
        .stack()
        .reset_index()
        .query("level_0 > level_1")
        .replace({"level_0": dict(enumerate(words)), "level_1": dict(enumerate(words))})
        .rename(columns={"level_0": "First", "level_1": "Second", 0: model_name.split("/")[-1]})
        .round(2)
    )
    return df

results = []
for m in models:
    embeddings = OllamaEmbeddings(model=m)
    results.append(compare_words(sentences, embeddings=embeddings, model_name=m))

result = pd.concat([results[0], results[1].iloc[:,-1].to_frame(), results[2].iloc[:,-1].to_frame(), results[3].iloc[:,-1].to_frame()], axis=1)
GT(
    result
).tab_header(
    "Cosine similarities between First and Second phrases"
).tab_spanner(
    label="model", 
    columns=[2,3,4,5]
).tab_style(
        style=style.fill(color="lightgreen"), locations=[loc.body(columns=[2,3,4], rows=[1])]
).tab_style(
    style=style.fill(color="lightgreen"), locations=loc.body(columns=[5], rows=[0])
)


GT(_tbl_data=                                         First  \
3  The woman in the dress became a politician.   
6          The man with the tie ran for a bus.   
7          The man with the tie ran for a bus.   

                                        Second  all-minilm  mxbai-embed-large  \
3         The man with the tie ran for office.        0.29               0.59   
6         The man with the tie ran for office.        0.64               0.69   
7  The woman in the dress became a politician.        0.09               0.31   

   nomic-embed-text  llama3.1  
3              0.64      0.91  
6              0.84      0.87  
7              0.50      0.85  , _body=<great_tables._gt_data.Body object at 0x13bb76c60>, _boxhead=Boxhead([ColInfo(var='First', type=<ColInfoTypeEnum.default: 1>, column_label='First', column_align='left', column_width=None), ColInfo(var='Second', type=<ColInfoTypeEnum.default: 1>, column_label='Second', column_align='left', column_width=None), ColInfo(var='all-minilm', type=<ColInfoTypeEnum.default: 1>, column_label='all-minilm', column_align='right', column_width=None), ColInfo(var='mxbai-embed-large', type=<ColInfoTypeEnum.default: 1>, column_label='mxbai-embed-large', column_align='right', column_width=None), ColInfo(var='nomic-embed-text', type=<ColInfoTypeEnum.default: 1>, column_label='nomic-embed-text', column_align='right', column_width=None), ColInfo(var='llama3.1', type=<ColInfoTypeEnum.default: 1>, column_label='llama3.1', column_align='right', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x10682bf80>, _spanners=Spanners([SpannerInfo(spanner_id='model', spanner_level=0, spanner_label='model', spanner_units=None, spanner_pattern=None, vars=['all-minilm', 'mxbai-embed-large', 'nomic-embed-text', 'llama3.1'], built=None)]), _heading=Heading(title='Cosine similarities between First and Second phrases', subtitle=None, preheader=None), _stubhead=None, _source_notes=[], _footnotes=[], _styles=[StyleInfo(locname=LocBody(columns=[2, 3, 4], rows=[1]), grpname=None, colname='all-minilm', rownum=1, colnum=None, styles=[CellStyleFill(color='lightgreen')]), StyleInfo(locname=LocBody(columns=[2, 3, 4], rows=[1]), grpname=None, colname='mxbai-embed-large', rownum=1, colnum=None, styles=[CellStyleFill(color='lightgreen')]), StyleInfo(locname=LocBody(columns=[2, 3, 4], rows=[1]), grpname=None, colname='nomic-embed-text', rownum=1, colnum=None, styles=[CellStyleFill(color='lightgreen')]), StyleInfo(locname=LocBody(columns=[5], rows=[0]), grpname=None, colname='llama3.1', rownum=0, colnum=None, styles=[CellStyleFill(color='lightgreen')])], _locale=<great_tables._gt_data.Locale object at 0x13bb75d90>, _formats=[], _substitutions=[], _options=Options(table_id=OptionsInfo(scss=False, category='table', type='value', value=None), table_caption=OptionsInfo(scss=False, category='table', type='value', value=None), table_width=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_layout=OptionsInfo(scss=True, category='table', type='value', value='fixed'), table_margin_left=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_margin_right=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_background_color=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_additional_css=OptionsInfo(scss=False, category='table', type='values', value=[]), table_font_names=OptionsInfo(scss=False, category='table', type='values', value=['-apple-system', 'BlinkMacSystemFont', 'Segoe UI', 'Roboto', 'Oxygen', 'Ubuntu', 'Cantarell', 'Helvetica Neue', 'Fira Sans', 'Droid Sans', 'Arial', 'sans-serif']), table_font_size=OptionsInfo(scss=True, category='table', type='px', value='16px'), table_font_weight=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_style=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_color=OptionsInfo(scss=True, category='table', type='value', value=

Llama is the only model that correctly identifies the man and woman going into politics as more similar to each other than the two men with the ties (one running for a bus and the other for office.)

My takeaways are: 

- If I were creating a semantic search, Llama would show me the correct result at the top.
- However, Llama's representation of a man running for a bus and a woman becoming a politician are quite close, so maybe I just lucked out.
- nomic-embed-text has the second highest score for the politicians being similar.

So, which model is the best one for embedding. The answer is probably: it depends. Sigh.